In [2]:
ls data

2012_before_coding.zip
Before coding.zip
Classified_Reviews_Tom.xlsx
Classified_Reviews_Tom_2012_2016.txt
Classified_Reviews_Tom_2012_2016_noformat.txt
Reviews_12_11_2016.csv
Reviews_2012_2013_Before_Coding.zip
Reviews_4_30_2016.csv
Reviews_6_16_2017.csv
Unbiased_Columbia_Yelp_Reviews_Labeled_3_3_17.csv
before coding/
before coding 2/
before coding 3/
biased.csv
unbiased.csv


In [50]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
sb.set()

### Data 

* 2012_before_coding.zip: folder containing all of the spreadsheets for reviews for which the classifier was run on in 2012
* Reviews_2012_2013_before_coding.zip: same as above, but covers 2013 as well
* Before_coding.zip: same as above, covering 2013-October 2015
* Reviews_4_30_2016: reviews for which the classifier was run on for October 2015-April 2016
* Reviews_12_11_2016: same as above, but through May 2016-12/11/2016
* Reviews_6_16_2017: same as above, but through today and including annotations
* For all reviews 12/11/2016 or before, the annotations are stored in Classified_Reviews_Tom.xslx (a txt file, I hope this is okay – let me know if not!)

### Plan

* We will load all reviews with annotations into one csv (biased)
* All others will be loaded into the unbiased dataset

### There are a couple of exceptions

* We drop all reviews that aren't Yes/No for the `is_foodborne` and `is_multiple`

In [4]:
labeled_cols = ['url', 'date', 'text', 'is_foodborne', 'is_multiple']
unlabeled_cols = ['url', 'date', 'text']
biased = pd.DataFrame(columns=labeled_cols)
unbiased = pd.DataFrame(columns=unlabeled_cols)

### Load in the labeled data

In [5]:
older_classified = pd.read_excel('data/Classified_Reviews_Tom.xlsx', )
older_classified.rename(columns = {'URL':'url', 'Date_Review':'date', 'Review':'text', 
                                   'Foodborne_Disease':'is_foodborne', '2_Or_More_Ill':'is_multiple'}, 
                        inplace = True)
older_classified['date'] = pd.to_datetime(older_classified['date'])

In [6]:
# remove bad rows
good_values = {'Yes', 'No'}
print '{} total rows'.format(len(older_classified))
older_classified = older_classified[older_classified['is_foodborne'].isin(good_values)]
older_classified = older_classified[older_classified['is_multiple'].isin(good_values)]
print '{} after removal'.format(len(older_classified))

14275 total rows
11308 after removal


In [7]:
biased = biased.append(older_classified[labeled_cols])

In [8]:
newer = pd.read_csv('data/Reviews_6_16_2017.csv')
newer.rename(columns={'multiple':'is_multiple', 'date_review':'date', 'review':'text'}, 
             inplace=True)
newer['date'] = pd.to_datetime(newer['date'], format='%d%b%Y:%H:%M:%S.000')
newer['is_foodborne'] = newer['is_foodborne'].map({'YES':'Yes', 'NO':'No'})
newer['is_multiple'] = newer['is_multiple'].map({'YES':'Yes', 'NO':'No'})

# remove bad rows
good_values = {'Yes', 'No'}
print '{} total rows'.format(len(newer))
newer_classified = newer[newer['is_foodborne'].isin(good_values) & newer['is_multiple'].isin(good_values)]
newer_unclassified = newer[newer['is_foodborne'].isnull()]
# I check, all null `is_foodborne`s are null `is_multiple`s also
print '{} labeled after removal'.format(len(newer_classified))
print '{} unlabeled after removal'.format(len(newer_unclassified))

412829 total rows
2251 labeled after removal
410539 unlabeled after removal


In [9]:
biased = biased.append(newer_classified[labeled_cols])
unbiased = unbiased.append(newer_unclassified[unlabeled_cols])

### Load in the rest of the unlabeled data

In [10]:
csv = pd.read_csv('data/Reviews_4_30_2016.csv')
csv.rename(columns={'date_review':'date', 'review':'text'}, 
           inplace=True)
# drop bad rows and parse dates
print '{} rows before'.format(len(csv))
csv = csv[~csv['zip'].isnull()]
print '{} rows after'.format(len(csv))
csv['date'] = pd.to_datetime(csv['date'], format='%d%b%Y:%H:%M:%S.000')

441280 rows before
377758 rows after


In [11]:
unbiased = unbiased.append(csv[unlabeled_cols])

In [12]:
csv = pd.read_csv('data/Reviews_12_11_2016.csv')
csv.rename(columns={'date_review':'date', 'review':'text'}, 
           inplace=True)
# drop bad rows and parse dates
print '{} rows before'.format(len(csv))
csv = csv[~csv['zip'].isnull()]
print '{} rows after'.format(len(csv))
csv['date'] = pd.to_datetime(csv['date'], format='%d%b%Y:%H:%M:%S.000')

498138 rows before
415148 rows after


In [13]:
unbiased = unbiased.append(csv[unlabeled_cols])
print '{} total reviews'.format(len(unbiased))

1203445 total reviews


In [14]:
for d in ['before coding', 'before coding 2', 'before coding 3']:
    for f in os.listdir('data/'+d):
#         print(f)
        if 'xls' in f:
            csv = pd.read_excel('data/'+d+'/'+f)
        else:
            try:
                csv = pd.read_csv('data/'+d+'/'+f)
            except:
                print('ERROR: {}'.format(f))
                continue
            
        csv.rename(columns={'Date':'date', 'Review':'text', 'URL':'url'}, 
                   inplace=True)
        # drop bad rows and parse dates
#         print f
#         print csv.head(1)
        print '{} rows before'.format(len(csv)),
        csv = csv[~csv['Score'].isnull()]
        print '{} rows after'.format(len(csv))
        csv['date'] = pd.to_datetime(csv['date'])
        unbiased = unbiased.append(csv[unlabeled_cols])

259 rows before 259 rows after
61 rows before 61 rows after
61 rows before 61 rows after
266 rows before 266 rows after
23 rows before 23 rows after
2 rows before 2 rows after
13 rows before 13 rows after
6 rows before 6 rows after
3 rows before 3 rows after
32 rows before 32 rows after
8663 rows before 8663 rows after
8492 rows before 8492 rows after
19 rows before 17 rows after
14 rows before 14 rows after
20 rows before 20 rows after
17 rows before 17 rows after
17 rows before 17 rows after
36 rows before 36 rows after
18 rows before 18 rows after
16 rows before 16 rows after
7 rows before 7 rows after
7 rows before 7 rows after
19 rows before 19 rows after
ERROR: yelp_analysis-2012-10-10-092629.csv
20 rows before 20 rows after
29 rows before 29 rows after
13 rows before 13 rows after
4 rows before 4 rows after
11 rows before 11 rows after
63 rows before 63 rows after
42 rows before 42 rows after
33 rows before 33 rows after
56 rows before 56 rows after
42 rows before 42 rows after


### Clean up the final dataframes

In [39]:
# fix up an encoding mismatch
biased['text'] = biased['text'].map(lambda x:x.decode('ISO-8859-1') if type(x) is str else x)
unbiased['text'] = unbiased['text'].map(lambda x:x.decode('ISO-8859-1') if type(x) is str else x)

In [16]:
# remove all reviews that are not labeled from the unbiased or some unbiased nulls
print '{} biased before dupe removal'.format(len(biased))
biased = biased.drop_duplicates()
print '{} biased after dupe removal'.format(len(biased))

print '{} unbiased before null removal'.format(len(unbiased))
unbiased = unbiased[~unbiased['text'].isnull()]
print '{} unbiased after null removal'.format(len(unbiased))
unbiased = unbiased.drop_duplicates()
print '{} unbiased after dupe removal'.format(len(unbiased))

13559 biased before dupe removal
13541 biased after dupe removal
1261097 unbiased before null removal
1217243 unbiased after null removal
1178842 unbiased after dupe removal


In [17]:
print '{} unbiased before removing biased'.format(len(unbiased))
biased_set = { tuple(t) for t in biased[unlabeled_cols].values.tolist() }
in_biased = unbiased.apply(lambda row:tuple(row.values) in biased_set, axis=1)
unbiased = unbiased[~in_biased]

1178842 unbiased before removing biased


In [18]:
print '{} unbiased after removing biased'.format(len(unbiased))

1176028 unbiased after removing biased


### Write out the final csvs

In [41]:
biased.to_csv('data/biased.csv', header=True, index=False, encoding="utf8")
unbiased.to_csv('data/unbiased.csv', header=True, index=False, encoding="utf8")

In [37]:
print 'All done. Final example counts:'
print '{} labeled reviews'.format(len(biased))
print '{} unlabeled reviews'.format(len(unbiased))

All done. Final example counts:
13541 labeled reviews
1176028 unlabeled reviews


### Also create a sample csv for labeling from the unbiased

In [41]:
split_date = datetime.datetime.strptime('1/1/2017', '%d/%m/%Y')
new_unbiased = unbiased[unbiased.date >= split_date]
sample = new_unbiased.sample(1000)
sample['is_foodborne'] = np.nan
sample['is_multiple'] = np.nan
sample.to_excel('data/new_unbiased_sample_to_label.xlsx', index=False, header=True, encoding='utf8')